TASK 1 - 2

In [ ]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np

# load target
target = np.load("target_1.npy")

# define time horizon
T = len(target[0])

# define system dynamics
A = np.array([[1, 0, 0.1, 0], [0, 1, 0, 0.1], [0, 0, 0.8, 0], [0, 0, 0, 0.8]])

B = np.array([[0, 0], [0, 0], [0.1, 0], [0, 0.1]])

E = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])

# create cp variables
x = cp.Variable((4, T))
u = cp.Variable((2, T - 1))


# Tracking Error
TE = cp.sum([cp.norm(E @ x[:, t] - target[:, t], 2) for t in range(T)])

# Control Effort
CE = cp.sum_squares(u[:, : T - 1])

# define constraints
constraints = [x[:, 0] == np.array([0.5, 0, 1, -1])]

for t in range(T - 1):
    constraints += [x[:, t + 1] == A @ x[:, t] + B @ u[:, t]]

# result storage
results = []

TE_best = []
CE_best = []

# define objective
rho = 10
for rho in [10, 5, 2, 1, 0.5, 0.1, 0.05, 0.02, 0.01, 0.005, 0.002]:
    objective = cp.Minimize(TE + rho * CE)
    prob = cp.Problem(objective, constraints)
    prob.solve()
    results.append((rho, x[0].value, x[1].value, TE.value, CE.value))

    # print("x[0] = ", x[0].value,"\nx[1] = ", x[1].value)
    # print(TE.value, CE.value)



# plot results
for rho, x1, x2, _, _ in results:
    plt.scatter(x1, x2, label=f"rho={rho}")  # scatter is better for points

# plot target as a red star
plt.scatter(target[0], target[1], color="red", marker="*", s=150, label="target")

plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Solutions for different rho values")
plt.legend()
plt.savefig("solutions_plot.png", dpi=500)  # can also use .pdf, .svg, etc.

plt.show()




# plot for different rho values with TE and CE components
for rho, _, _, TE, CE in results:
    plt.scatter(TE, CE, label=f"rho={rho}")  # scatter is better for points

plt.xlabel("TE")
plt.ylabel("CE")
plt.title("TE and CE for different rho values")
plt.legend()
plt.show()



